In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
%matplotlib inline

### Calculating VIF

In [2]:
swiss = pd.read_csv('../data/swiss.csv', ).drop('Unnamed: 0', axis=1)
swiss.rename(columns={'Infant.Mortality': 'Infant_Mortality'}, inplace=True)

In [3]:
all_predictors = swiss.columns.difference(['Fertility'])
formula_all = 'Fertility ~ ' + '+'.join(all_predictors)
formula_all

'Fertility ~ Agriculture+Catholic+Education+Examination+Infant_Mortality'

In [4]:
lm = smf.ols(formula_all, data=swiss).fit()

In [5]:
X = sm.add_constant(swiss.drop('Fertility', axis=1)).rename(columns={'const': 'Intercept'})
X_col = X.columns
X_val = X.values

/Users/seungjaebang/Envs/sp_academy/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [6]:
data = []

for i in range(1, X.shape[1]):
    vif = variance_inflation_factor(X_val, i)
    data.append({'feature': X_col[i], 'vif': vif})

In [7]:
vifs = pd.DataFrame(data)
vifs['vif_sqrt'] = np.sqrt(vifs['vif'])

In [8]:
vifs

,feature,vif,vif_sqrt
0,Agriculture,2.284129,1.511334
1,Examination,3.675420,1.917138
2,Education,2.774943,1.665816
3,Catholic,1.937160,1.391819
4,Infant_Mortality,1.107542,1.052398


### ANOVA - for Including Predictors

In [9]:
lm1 = smf.ols('Fertility ~ Agriculture', data=swiss).fit()
lm2 = smf.ols('Fertility ~ Agriculture + Examination + Education', data=swiss).fit()
lm3 = smf.ols(formula_all, data=swiss).fit()

In [10]:
sm.stats.anova_lm(lm1, lm2, lm3)

/Users/seungjaebang/Envs/sp_academy/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/seungjaebang/Envs/sp_academy/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/seungjaebang/Envs/sp_academy/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
0,45.0,6283.115793,0.0,NaN,NaN,NaN
1,43.0,3180.924879,2.0,3102.190915,30.210744,6.389104e-09
2,41.0,2105.042930,2.0,1075.881948,10.477497,2.111080e-04
